In [1]:
import sys
sys.path.append('../')
import config
import tweepy
import os
import json
import codecs

import pandas as pd
import datetime
from datetime import datetime, timedelta
import locale

In [2]:
# Clientを返す認証関数
def auth():
    # API KEY
    CK = config.CK
    CS = config.CS
    AT = config.AT
    AS = config.AS
    BT = config.BT

    # tweepy setting
    # auth = tweepy.OAuthHandler(CK, CS)
    # auth.set_access_token(AT, AS)
    

    # wait_on_rate_limit = True とするとレートリミットを確認しながら取得処理を行う
    client = tweepy.Client(bearer_token = BT,\
                           consumer_key = CK,\
                           consumer_secret = CS,\
                           access_token = AT,\
                           access_token_secret = AS,\
                           wait_on_rate_limit = True)

    return client

In [3]:
# Use tweepy.Client()
client = auth()

In [4]:
# Define a search_term for searching tweets
search_term = "CVE"

In [5]:
# 取得したいツイート数
limit = 50000  

In [11]:
# https://qiita.com/daisuke-aoki/items/30e3d6e84637326ef1da
# 取得対象のツイートの時間幅を指定する この例では実行前の6日間（日曜日に実行すると、月曜から土曜までを取得
# iso形式のUTC時間で指定しないと正しく時間指定ができない模様。
# 指定した時間幅に、limitで指定した件数以上のツイートがあってもlimit以上は取得しない
now = datetime.now()
print(now)
start_time = now - timedelta(days=6)
start_time = start_time.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(start_time)
end_time = now.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(end_time)
# now = now.replace(minute=0, second=0, microsecond=0)
end_time_tweepy = str(end_time.isoformat()) +'+09:00'
# start_time = now - timedelta(days=7) 
start_time_tweepy = str(start_time.isoformat())+'+09:00'
print(start_time_tweepy)
print(end_time_tweepy)

2023-02-19 17:53:13.242212
2023-02-13 03:00:00
2023-02-19 00:00:00
2023-02-13T03:00:00+09:00
2023-02-19T00:00:00+09:00


In [12]:
df_tweet = pd.DataFrame()
for tweet in tweepy.Paginator(client.search_recent_tweets,\
                              query = search_term,\
                              start_time=start_time_tweepy,\
                              end_time=end_time_tweepy,\
                              tweet_fields=['id','created_at','text','author_id','public_metrics',],\
                              max_results = 100).flatten(limit = limit):
    df_tweet = pd.concat([df_tweet, pd.DataFrame([tweet.data])])

print(df_tweet)

                                                 text            author_id  \
0   AMD Ryzen Masterに脆弱性。システムが完全に制御される恐れ。2.10.1.22...            155942328   
0   💰💰💰💰💰💰💰💰💰💰💰💰\n🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥\n\nTrade-Ideas is ...  1527831839255285760   
0   RT @Horizon3Attack: CVE-2022-39952, announced ...  1620281239587495939   
0   🚨 NEW: CVE-2023-22246 🚨 Adobe Animate versions...   904129001723011072   
0   RT @ldsopreload: someone has made basically an...   892148661609746432   
..                                                ...                  ...   
0   New post from https://t.co/uXvPWJy6tj (CVE-202...   955014888446939136   
0   New post from https://t.co/9KYxtdZjkl (CVE-202...   958005194398289920   
0   New post from https://t.co/uXvPWJy6tj (CVE-202...   955014888446939136   
0   New post from https://t.co/9KYxtdZjkl (CVE-202...   958005194398289920   
0   New post from https://t.co/uXvPWJy6tj (CVE-202...   955014888446939136   

                     id                created_at  \
0   162695

In [13]:
filename = start_time.strftime("%Y%m%d") + "-" + end_time.strftime("%Y%m%d") 
print(filename)

20230213-20230219


In [14]:
df_tweet.to_csv(filename + ".csv", index=False,header=False)

In [19]:
df = pd.read_csv('20230213-20230219.csv', header = None, names = ["text", "author_id","id","created_at", "public_metrics","edit_history_tweet_ids"])
df.head()
df.tail()

,text,author_id,id,created_at,public_metrics,edit_history_tweet_ids
18356,New post from https://t.co/uXvPWJy6tj (CVE-202...,955014888446939136,1624830774565392390,2023-02-12T18:00:19.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624830774565392390']
18357,New post from https://t.co/9KYxtdZjkl (CVE-202...,958005194398289920,1624830758425616385,2023-02-12T18:00:15.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624830758425616385']
18358,New post from https://t.co/uXvPWJy6tj (CVE-202...,955014888446939136,1624830756378869763,2023-02-12T18:00:14.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624830756378869763']
18359,New post from https://t.co/9KYxtdZjkl (CVE-202...,958005194398289920,1624830739924525057,2023-02-12T18:00:10.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624830739924525057']
18360,New post from https://t.co/uXvPWJy6tj (CVE-202...,955014888446939136,1624830737726816263,2023-02-12T18:00:10.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624830737726816263']


In [20]:
df.head()

,text,author_id,id,created_at,public_metrics,edit_history_tweet_ids
0,AMD Ryzen Masterに脆弱性。システムが完全に制御される恐れ。2.10.1.22...,155942328,1626959611906658304,2023-02-18T14:59:33.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626959611906658304']
1,💰💰💰💰💰💰💰💰💰💰💰💰\n🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥\n\nTrade-Ideas is ...,1527831839255285760,1626959438325116931,2023-02-18T14:58:52.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626959438325116931']
2,"RT @Horizon3Attack: CVE-2022-39952, announced ...",1620281239587495939,1626959335342448640,2023-02-18T14:58:27.000Z,"{'retweet_count': 298, 'reply_count': 0, 'like...",['1626959335342448640']
3,🚨 NEW: CVE-2023-22246 🚨 Adobe Animate versions...,904129001723011072,1626958943468548099,2023-02-18T14:56:54.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626958943468548099']
4,RT @ldsopreload: someone has made basically an...,892148661609746432,1626958932961787904,2023-02-18T14:56:51.000Z,"{'retweet_count': 47, 'reply_count': 0, 'like_...",['1626958932961787904']


In [21]:
# The number of tweets between 0213-0219(JST)
len(df)

18361

In [22]:
df_matched = df[df["text"].str.match('CVE-[0-9]{4}-[0-9]{4,5}')]
# The number of tweets matched with the CVE number form
len(df_matched)

2948

In [23]:
df_matched.tail()

,text,author_id,id,created_at,public_metrics,edit_history_tweet_ids
17956,CVE-2020-36661 A vulnerability was found in Ko...,821806287461740544,1624887400664076289,2023-02-12T21:45:19.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624887400664076289']
18031,CVE-2020-36661 : A vulnerability was found in ...,3183315712,1624877714908844038,2023-02-12T21:06:50.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624877714908844038']
18101,CVE-2015-10079 | juju2143 WalrusIRC 0.0.2 publ...,1416798413937643521,1624867321683791872,2023-02-12T20:25:32.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624867321683791872']
18252,CVE-2022-4905 | UDX Stateless Media Plugin 3.1...,1416798413937643521,1624844293973372928,2023-02-12T18:54:02.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624844293973372928']
18342,CVE-2023-0788 is called Blotchy Ptarmigan\nhtt...,1096056138569842688,1624831138853228545,2023-02-12T18:01:46.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624831138853228545']


In [24]:
df_matched.head()

,text,author_id,id,created_at,public_metrics,edit_history_tweet_ids
49,CVE-2023-0908 shall henceforth be named Reserv...,1096056138569842688,1626945412203511809,2023-02-18T14:03:08.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626945412203511809']
62,CVE-2023-0902 is called Seaworthy Osprey\nhttp...,1096056138569842688,1626942359475744768,2023-02-18T13:51:00.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626942359475744768']
70,CVE-2022-32132\n\nCVE-2022-32132\nReflected XS...,941389496771399680,1626941063368679427,2023-02-18T13:45:51.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626941063368679427']
74,CVE-2022-31889\n\nCVE-2022-31889\nReflected XS...,941389496771399680,1626940307366445056,2023-02-18T13:42:51.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626940307366445056']
82,CVE-2023-0907 shall henceforth be named Pregna...,1096056138569842688,1626938418746859521,2023-02-18T13:35:20.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626938418746859521']


In [26]:
# Extract only the CVE from the matched columns

import re
list_CVE = []
for i, s in enumerate(df_matched["text"].to_list()):
    m = re.search('CVE-[0-9]{4}-[0-9]{4,5}',s)
    list_CVE.append(m.group())

In [27]:
print(len(list_CVE))
print(list_CVE[:5])
print(list_CVE[-5:])

2948
['CVE-2023-0908', 'CVE-2023-0902', 'CVE-2022-32132', 'CVE-2022-31889', 'CVE-2023-0907']
['CVE-2020-36661', 'CVE-2020-36661', 'CVE-2015-10079', 'CVE-2022-4905', 'CVE-2023-0788']


In [30]:
index_list = [i for i in df_matched.index]

In [31]:
pd.DataFrame({'CVE': list_CVE}, index = index_list).head()

,CVE
49,CVE-2023-0908
62,CVE-2023-0902
70,CVE-2022-32132
74,CVE-2022-31889
82,CVE-2023-0907


In [32]:
pd.DataFrame({'CVE': list_CVE}, index = index_list).tail()

,CVE
17956,CVE-2020-36661
18031,CVE-2020-36661
18101,CVE-2015-10079
18252,CVE-2022-4905
18342,CVE-2023-0788


In [33]:
import numpy as np
df_plusCVE = pd.concat([df_matched, pd.DataFrame({'CVE': list_CVE}, index = index_list)], axis = 1)
df_plusCVE.head()

,text,author_id,id,created_at,public_metrics,edit_history_tweet_ids,CVE
49,CVE-2023-0908 shall henceforth be named Reserv...,1096056138569842688,1626945412203511809,2023-02-18T14:03:08.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626945412203511809'],CVE-2023-0908
62,CVE-2023-0902 is called Seaworthy Osprey\nhttp...,1096056138569842688,1626942359475744768,2023-02-18T13:51:00.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626942359475744768'],CVE-2023-0902
70,CVE-2022-32132\n\nCVE-2022-32132\nReflected XS...,941389496771399680,1626941063368679427,2023-02-18T13:45:51.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626941063368679427'],CVE-2022-32132
74,CVE-2022-31889\n\nCVE-2022-31889\nReflected XS...,941389496771399680,1626940307366445056,2023-02-18T13:42:51.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626940307366445056'],CVE-2022-31889
82,CVE-2023-0907 shall henceforth be named Pregna...,1096056138569842688,1626938418746859521,2023-02-18T13:35:20.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1626938418746859521'],CVE-2023-0907


In [34]:
df_plusCVE.tail()

,text,author_id,id,created_at,public_metrics,edit_history_tweet_ids,CVE
17956,CVE-2020-36661 A vulnerability was found in Ko...,821806287461740544,1624887400664076289,2023-02-12T21:45:19.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624887400664076289'],CVE-2020-36661
18031,CVE-2020-36661 : A vulnerability was found in ...,3183315712,1624877714908844038,2023-02-12T21:06:50.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624877714908844038'],CVE-2020-36661
18101,CVE-2015-10079 | juju2143 WalrusIRC 0.0.2 publ...,1416798413937643521,1624867321683791872,2023-02-12T20:25:32.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624867321683791872'],CVE-2015-10079
18252,CVE-2022-4905 | UDX Stateless Media Plugin 3.1...,1416798413937643521,1624844293973372928,2023-02-12T18:54:02.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624844293973372928'],CVE-2022-4905
18342,CVE-2023-0788 is called Blotchy Ptarmigan\nhtt...,1096056138569842688,1624831138853228545,2023-02-12T18:01:46.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1624831138853228545'],CVE-2023-0788


In [35]:
len(df_plusCVE["CVE"].unique())

866

In [36]:
# https://stackoverflow.com/questions/40454030/count-and-sort-with-pandas
# https://stackoverflow.com/questions/41415017/count-unique-values-using-pandas-groupby
df_ids = df_plusCVE[["author_id", "CVE"]].groupby(["CVE"])["author_id"].nunique()
df_ids

CVE
CVE-2012-10007    1
CVE-2014-12508    1
CVE-2014-2227     1
CVE-2014-4607     1
CVE-2015-10079    5
                 ..
CVE-2023-25784    1
CVE-2023-25786    1
CVE-2023-25796    1
CVE-2023-25797    1
CVE-2023-26020    5
Name: author_id, Length: 866, dtype: int64

In [37]:
# https://stackoverflow.com/questions/40454030/count-and-sort-with-pandas
df_ids_sorted = df_ids.reset_index(name = 'count').sort_values(['count'], ascending = False)
df_ids_sorted

,CVE,count
668,CVE-2023-23752,15
481,CVE-2023-20032,13
660,CVE-2023-23529,11
699,CVE-2023-23947,10
421,CVE-2023-0662,10
...,...,...
730,CVE-2023-24486,1
176,CVE-2022-33232,1
175,CVE-2022-33229,1
174,CVE-2022-33225,1


In [38]:
df_over10 = df_ids_sorted[df_ids_sorted['count']>= 10]
df_over10

,CVE,count
668,CVE-2023-23752,15
481,CVE-2023-20032,13
660,CVE-2023-23529,11
699,CVE-2023-23947,10
421,CVE-2023-0662,10
